In [26]:
using SymEngine
using Expr2LaTeX
using MacroTools

In [2]:
matrix_sym(name, nrows, ncols) = [symbols(Symbol(name,:_,i,:_,j)) for i in 1:nrows, j in 1:ncols]

matrix_sym (generic function with 1 method)

In [3]:
"Converts a SymEngine expression into a julia one"
function juliaize(symexpr)
    str = repr(symexpr)
    str = replace(str, "SymEngine.Basic", "")
    str = replace(str, r"(.)_(\d+)_(\d+)", s"\g<1>[\g<2>, \g<3>]")
    parse(str)
end

juliaize

In [102]:
Expr(:+,1,2,34) 

:($(Expr(:+, 1, 2, 34)))

In [143]:
rewrite(nonexpr, op, cutoff) = nonexpr
function rewrite(expr::Expr, op, cutoff)
    if expr.head==:call && expr.args[1]==op && length(expr.args)-1>cutoff
        Expr(:call, op, 
        Expr(:call, op, expr.args[2:cutoff+1]...),
        Expr(:call, op, expr.args[cutoff+1:end]...)
        )
    else
        expr
    end
end

rewrite_rule(op, cutoff) = expr -> rewrite(expr, op, cutoff)
group_addition(expr) = MacroTools.prewalk(rewrite_rule(:+, 11), expr)

group_addition (generic function with 1 method)

In [144]:
function matmul_sym(X1,X2,Y1,Y2)
    x = matrix_sym(:X, X1, X2)
    y = matrix_sym(:Y, Y1, Y2)
    x*y
end
matmul_sym(args...) = matmul_sym(Val.(args)...)

matmul(x,y) = matmul(x,y,Val.(size(x))..., Val.(size(y))...)

@generated function matmul(X, Y, ::Val{X1},::Val{X2}, ::Val{Y1}, ::Val{Y2}) where {X1,X2,Y1,Y2}
    r = juliaize(matmul_sym(X1,X2,Y1,Y2))
    quote
        @inbounds ret = $(r)
        ret
    end |> macroexpand |> rmlines |> unblock |> group_addition
end

matmul (generic function with 2 methods)

In [145]:
using BenchmarkTools
a, b  = (rand(Int8, 3,100), rand(Int8, 100,4))
@btime matmul($a, $b)
@btime *(a,b)

  53.163 μs (1 allocation: 96 bytes)
  2.254 μs (5 allocations: 320 bytes)


3×4 Array{Int8,2}:
  71  -15  -44  -93
 125    0   52   91
   3  -87  -20  119

In [146]:
;git commit -m="" -a
;  git push

LoadError: [91msyntax: extra token "commit" after end of expression[39m